In [1]:
from Ham import *
from Circ import *

import numpy as np
from numpy import linalg

import matplotlib
import matplotlib.pyplot as plt

from math import pi

from qiskit import *
from qiskit_aer import QasmSimulator, UnitarySimulator, StatevectorSimulator
from qiskit.quantum_info import Pauli


from qiskit.visualization import plot_bloch_multivector

#### Parametri sistema

In [2]:
# Numero spin anello
n_spin = 4

# Periodicià anello
chiuso = True

# Costanti d'interazione [in unità di J]
J = [1,1,1]

# Parametro per la regolazione della velocità del processo di annealing
lam = 0.1

# Funzione di drive per l'annealing
def drive(param, t):
#   Drive lineare
    ris = param * t

    return ris

# Funzione di drive per l'annealing del termine a singolo corpo
def drive_sp(param, n, t):
    ris = 0

    if (t * param) ** n < 1:
        ris = 1 - (t*param) ** n

    return ris

#### Creazione Hamiltoniana sistema

In [3]:
# Creiamo l'Hamiltoniana del sistema
H = crea_hamiltoniana(n_spin, chiuso)

#### Circuito per l'inizializzazione del sistema

In [4]:
# Creiamo il circuito per la preparazione dello stato iniziale
init = crea_circuito_inizializzazione(n_spin)
#init.draw()

#### Circuito per la digitalizzazione del processo di Annealing

In [5]:
# Numero di trotter step in cui digitalizzare il processo di annealing
n_tstep = 5

# Creaimo il circuito per l'esecuzione dell'annealing digitalizzato
digitalized_annealing = crea_circuito_annealing(J, n_spin, n_tstep, drive, lam, drive_sp, lam)
#digitalized_annealing.draw()

#### Unione Circuiti

In [6]:
# Uniamo i circuiti derivanti dall'inizializzazione e dal processo di annealing
final_circuit = init.compose(digitalized_annealing)
# final_circuit.draw()

#### Esecuzione Circuito tramite Statevector Simulator

In [7]:
# Allochiamo il backend opportuno, i.e. lo statevector_simulator
backend = StatevectorSimulator()

# Eseguiamo la simulazione e recuperiamo lo statevector del nostro sistema
# approssimazione del ground state
psi_fin = backend.run(final_circuit).result().get_statevector()

# ATTENZIONE:: per Qiskit |0> == |up> e |1> == |down>
# plot_bloch_multivector(psi_fin)

#### Calcolo Ground State Effettivo

In [8]:
# Valutazione Hamiltoniana di Heisenberg del sistema
H_fin = H.valuta_tot(J, drive, lam, 1/lam)

# Diagonalizzazione Hamiltoninana di Heisenberg
e, V = np.linalg.eigh(np.real(H_fin))

# Recuperiamo l'indice del groung state
id_gs = np.argmin(e)

# Recuperiamo il ground state e la relativa energia
gs   = V[:,id_gs]
e_gs = e[id_gs]

#### Calcolo di Overlap ed errore energia Ground State approssimato tramite Annealing digitale

In [9]:
# Calcolo overlap
overlap = abs(np.transpose(np.conjugate(gs)) @ np.array(psi_fin))**2
print(overlap)

# Calcolo errore-2 relativo per l'energia
err_e = abs(e_gs - np.transpose(np.conjugate(np.array(psi_fin))) @ H_fin @ np.array(psi_fin)) / abs(e_gs)
print(err_e)

[[0.98423833]]
[[0.01142435]]
